In [63]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [64]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [65]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Michał,Karwacki,63050131939,Bielsk Podlaski,ul. Szkolna 33/24,48324245029,michal.karwacki@kuba.pl,2014-01-01,None,menadżer,63.9
1,2,Mateusz,Pikora,91040994211,Wrocław,ul. Daliowa 14/26,48261537035,mateusz.pikora@kuba.pl,2014-01-01,None,sprzedawca,56.9
2,3,Marcin,Ślazyk,93012413357,Wrocław,ul. Plażowa 66/16,48991917947,marcin.slazyk@kuba.pl,2014-01-01,None,sprzedawca,51.5
3,4,Anna,Latko,87011958687,Wrocław,ul. Malinowa 54,48079898240,anna.latko@kuba.pl,2014-01-01,None,mechanik,55.1
10,5,Mateusz,Staszyk,91072865910,Wrocław,ul. Krótka 21/37,48183340274,mateusz.staszyk@kuba.pl,2014-01-01,None,informatyk,41.0
4,6,Krystian,Stańczak,83022438497,Szczecin,ul. Jaśminowa 44/19,48640643761,krystian.stanczak@kuba.pl,2016-06-11,None,mechanik,54.2
5,7,Barbara,Kasperuk,93083118227,Wrocław,ul. Leszczynowa 31/49,48907560149,barbara.kasperuk@kuba.pl,2016-12-22,None,mechanik,49.4
6,8,Urszula,Sokolova,78082097382,Wrocław,ul. Piaskowa 89/16,48425580517,urszula.sokolova@kuba.pl,2017-08-12,None,pomocnik mechanika,37.2
7,9,Tomasz,Nogowski,67030589653,Wrocław,ul. Pocztowa 32/10,48420674747,tomasz.nogowski@kuba.pl,2017-10-10,2017-12-28,diagnosta samochodowy,47.4
8,10,Ksawery,Lewicki,71122355790,Konstancin-Jeziorna,ul. Działkowa 90/48,48342304620,ksawery.lewicki@kuba.pl,2017-10-17,None,diagnosta samochodowy,47.8


In [66]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [67]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [68]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [69]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4]) for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [70]:
len(year_ceil), len(pd.unique(client_ids))

(1482, 1482)

In [71]:
year_ceil

[2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [72]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [73]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [74]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [75]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [76]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [77]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Inez,Dąbrowska,K,89031968792,Wrocław,ul. Pleszewska 73/18,True,48810361775,i.dabr89@outlook.com
1,2,Agnieszka,Wojtasik,K,98022852857,Wrocław,ul. Żwirki i Wigury 42/24,False,48269630579,wojtasi-agnie1998@onet.pl
2,3,Dominika,Tyszko,K,85010902812,Wrocław,ul. Wiklinowa 77,False,48240436823,d_tyszko@wp.pl
3,4,Wojciech,Janas,M,60021304550,Kraków,ul. Zamkowa 25/2,False,48933505936,janas19609@o2.pl
4,5,Szymon,Korolko,M,79021157075,Poznań,ul. Cedrowa 60/48,False,48188922797,s.korolko79@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1477,1478,Józef,Rogoziński,M,83022681512,Chęciny,ul. Opalowa 86/28,False,48317741044,r412rd49w7@opayq.com
1478,1479,Anna,Prekiel,K,92061562692,Wrocław,ul. Karczewska 14/39,False,48350773537,ann_prekie@wp.pl
1479,1480,Stanisław,Nowakowski,M,64111000654,Wrocław,ul. Unii Lubelskiej 83/41,False,48007709699,nowakowski19648@gmail.com
1480,1481,Krzysztof,Benbenek,M,93110209191,Piła,ul. Zielona 56/48,False,48887243345,k_benbenek1993@outlook.com


In [78]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [79]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Inez,Dąbrowska,K,89031968792,Wrocław,ul. Pleszewska 73/18,True,48810361775,i.dabr89@outlook.com
1,2,Agnieszka,Wojtasik,K,98022852857,Wrocław,ul. Żwirki i Wigury 42/24,False,48269630579,wojtasi-agnie1998@onet.pl
2,3,Dominika,Tyszko,K,85010902812,Wrocław,ul. Wiklinowa 77,False,48240436823,d_tyszko@wp.pl
3,4,Wojciech,Janas,M,60021304550,Kraków,ul. Zamkowa 25/2,False,48933505936,janas19609@o2.pl
4,5,Szymon,Korolko,M,79021157075,Poznań,ul. Cedrowa 60/48,False,48188922797,s.korolko79@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1477,1478,Józef,Rogoziński,M,83022681512,Chęciny,ul. Opalowa 86/28,False,48317741044,r412rd49w7@opayq.com
1478,1479,Anna,Prekiel,K,92061562692,Wrocław,ul. Karczewska 14/39,False,48350773537,ann_prekie@wp.pl
1479,1480,Stanisław,Nowakowski,M,64111000654,Wrocław,ul. Unii Lubelskiej 83/41,False,48007709699,nowakowski19648@gmail.com
1480,1481,Krzysztof,Benbenek,M,93110209191,Piła,ul. Zielona 56/48,False,48887243345,k_benbenek1993@outlook.com


In [80]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [81]:
np.sum(clients_df['karta_lojalnościowa'])

75

In [82]:
np.sum(pd.isna(clients_df["PESEL"]))

59

# auta

In [83]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [84]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [85]:
df_samochody = create_car_table(max(car_ids))

In [86]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Opel,Zafira,4,2014,Coupe,False,brązowy,Benzyna,2.6,163,Manualna,3,5,False
1,2,Peugeot,108,4,2003,SUV,False,czarny,Benzyna,1.3,72,Manualna,3,5,False
2,3,SEAT,Ibiza,4,2009,SUV,False,brązowy,Benzyna,1.8,110,Automatyczna,5,5,False
3,4,Renault,Twingo,4,2016,Hatchback,True,biały,Benzyna,1.5,92,Manualna,5,5,False
4,5,Renault,Twingo,4,2009,SUV,False,biały,Benzyna,1.1,69,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,963,Dacia,Sandero,4,2003,Hatchback,False,czarny,Benzyna,1.4,90,Manualna,5,5,True
963,964,Skoda,Octavia,4,2014,SUV,False,niebieski,Benzyna,1.7,110,Manualna,5,5,True
964,965,Audi,A6,4,2004,Sedan,False,czarny,Diesel,3.2,218,Automatyczna,5,5,True
965,966,Volkswagen,Touran,4,2003,SUV,False,szary,Benzyna,2.4,150,Manualna,5,5,False


In [87]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [88]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna'], dtype=object)

# Usługi

In [89]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [90]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [91]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [92]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2004' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [93]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [94]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,57600.0
1,2,2,2,2,2014-01-02,23700.0
2,3,3,3,3,2014-01-03,44000.0
3,4,4,4,2,2014-01-03,25700.0
4,5,5,5,2,2014-01-05,25800.0
...,...,...,...,...,...,...
932,933,964,1479,3,2017-12-28,61200.0
933,934,965,30,3,2017-12-28,59700.0
934,935,148,285,2,2017-12-29,31100.0
935,936,966,1481,3,2017-12-30,41600.0


In [95]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [96]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [97]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():
        sell_prices[i] *= 0.97

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1097819302.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():


[12, 1, 12, 6, 1, 18, 6, 6, 12, 6, 12, 12, 1, 6, 12, 1, 12, 18, 6, 18, 18, 18, 1, 12, 18, 6, 1, 1, 18, 18, 18, 6, 18, 12, 12, 18, 1, 12, 6, 18, 1, 6, 1, 12, 6, 1, 6, 6, 1, 12, 12, 12, 12, 6, 1, 1, 1, 6, 1, 18, 1, 1, 1, 6, 1, 6, 1, 18, 1, 12, 6, 6, 18, 6, 1, 12, 6, 6, 6, 1, 12, 12, 1, 18, 6, 1, 12, 1, 18, 1, 12, 1, 1, 6, 1, 6, 18, 1, 6, 18, 6, 18, 6, 1, 12, 1, 1, 6, 6, 1, 1, 1, 6, 1, 1, 18, 1, 12, 12, 6, 1, 18, 6, 1, 12, 1, 12, 12, 1, 1, 1, 12, 6, 12, 6, 1, 18, 6, 1, 18, 1, 1, 12, 1, 6, 6, 12, 1, 6, 12, 1, 18, 6, 12, 18, 12, 6, 1, 12, 6, 6, 18, 12, 1, 1, 18, 1, 6, 6, 6, 12, 1, 12, 12, 6, 1, 1, 6, 12, 1, 18, 6, 1, 1, 12, 12, 1, 12, 18, 12, 12, 1, 1, 1, 18, 6, 1, 1, 12, 1, 6, 1, 6, 6, 18, 1, 6, 1, 18, 1, 1, 1, 6, 18, 1, 18, 18, 1, 12, 12, 1, 1, 6, 6, 18, 1, 18, 1, 6, 6, 12, 1, 1, 1, 1, 12, 18, 18, 6, 18, 1, 12, 6, 1, 1, 1, 12, 1, 6, 6, 1, 18, 1, 12, 6, 1, 18, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 18, 1, 12, 12, 1, 1, 1, 1, 18, 12, 1, 18, 1, 1, 6, 1, 6, 12, 1, 1, 12, 12, 12, 12, 6, 18, 1, 6, 1, 12

In [98]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,14,21,3,53248.0,12,2014-01-14
1,2,17,22,2,95600.0,1,2014-01-15
2,3,4,23,2,39728.0,12,2014-01-15
3,4,16,24,2,43044.0,6,2014-01-16
4,5,3,25,3,59267.0,1,2014-01-16
...,...,...,...,...,...,...,...
925,926,499,609,2,33000.0,1,2017-12-24
926,927,145,1478,3,61098.0,6,2017-12-27
927,928,60,1351,2,67704.0,12,2017-12-29
928,929,148,1480,2,48000.0,1,2017-12-29


In [99]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [100]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [101]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(26, array([24], dtype=int64)), (37, array([5], dtype=int64)), (56, array([1], dtype=int64)), (58, array([15], dtype=int64)), (61, array([16], dtype=int64)), (68, array([23, 55], dtype=int64)), (85, array([ 3, 32], dtype=int64)), (96, array([4], dtype=int64)), (101, array([44, 57,  7], dtype=int64)), (105, array([38], dtype=int64)), (110, array([13], dtype=int64)), (112, array([14, 54], dtype=int64)), (113, array([57], dtype=int64)), (123, array([48], dtype=int64)), (124, array([45, 29, 13], dtype=int64)), (126, array([8], dtype=int64)), (127, array([20], dtype=int64)), (141, array([0], dtype=int64)), (151, array([21, 17], dtype=int64)), (154, array([41], dtype=int64)), (155, array([6], dtype=int64)), (160, array([60, 52, 15], dtype=int64)), (162, array([35], dtype=int64)), (184, array([59], dtype=int64)), (192, array([44], dtype=int64)), (205, array([50], dtype=int64)), (207, array([19], dtype=int64)), (212, array([61], dtype=int64)), (223, array([20], dtype=int64)), (224, array([58]

In [102]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [103]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Oświetlenie boczne,Honda Aviator,37,1,1
1,Opona zimowa,R15,388,2,4
2,Zderzak,Mercedes-Benz GLC 350 brązowy,647,3,1
3,Radio,Volvo V90,299,4,1
4,Głośnik,BMW 318,110,5,1
...,...,...,...,...,...
189,Skrzynia biegów,Volkswagen Golf Plus Automatyczna,1822,130,1
190,Filtr paliwa,Hero Pleasure,99,131,1
191,Felga,R16 biały,26,131,4
192,Bezpiecznik,,10,132,1


# Wyposażenie

In [104]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [105]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Oświetlenie boczne,Honda Aviator,37.0
1,2,Opona zimowa,R15,388.0
2,3,Zderzak,Mercedes-Benz GLC 350 brązowy,647.0
3,4,Radio,Volvo V90,299.0
4,5,Głośnik,BMW 318,110.0
...,...,...,...,...
188,155,Zbiornik paliwa,Volkswagen Golf Plus,406.0
189,156,Skrzynia biegów,Volkswagen Golf Plus Automatyczna,1822.0
190,157,Filtr paliwa,Hero Pleasure,99.0
191,158,Felga,R16 biały,26.0


# Usługi - uzupełnienie

In [106]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [107]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,26,51,7,2014-02-07,40
1,2,37,69,4,2014-02-25,1800
2,3,56,106,6,2014-03-19,750
3,4,58,108,7,2014-03-20,350
4,5,61,114,7,2014-03-25,130
...,...,...,...,...,...,...
128,129,905,370,6,2017-09-07,60
129,130,925,1426,4,2017-10-11,2960
130,131,946,1454,6,2017-11-21,240
131,132,950,1459,4,2017-11-24,10


In [108]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [109]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [110]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,2,2,4
2,3,3,3,1
3,4,4,4,1
4,5,5,5,1
...,...,...,...,...
189,190,130,156,1
190,191,131,157,1
191,192,131,158,4
192,193,132,150,1


In [111]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [112]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [113]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,215.0,7.0
1,2.0,2.0,4468.0,15.0
2,3.0,3.0,1135.0,2.0
3,4.0,4.0,1531.0,6.0
4,5.0,5.0,675.0,8.0
...,...,...,...,...
154,155.0,155.0,1457.0,4.0
155,156.0,156.0,1583.0,1.0
156,157.0,157.0,307.0,4.0
157,158.0,158.0,120.0,6.0


In [114]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv")

In [115]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [116]:
df_equipment.to_csv("outputs/wyposażenie.csv")